In [6]:
import nbformat  # module used for reading/writing jupyter notebooks
# nbvonvert.preprocessors is a submodule for executing notebooks
# executepreprocessors runs all the cells in a jupyter notebook
from nbconvert.preprocessors import ExecutePreprocessor  # Executes the code cells in a notebook
import json # for reading and writing or encode and decode json data
import torch  # this library is used for building and training neural networks
import torch.nn as nn # contains neural network layers, loss functions etc
import torch.optim as optim # module to optimization algorithm

In [7]:
# defining a class named rewardmodel which inherits from nn.module
# nn.module is the base class for all pytorch models
class RewardModel(nn.Module):
    def __init__(self, input_dim): # __init__ is a special method that runs when an object is created
        # self is a reference to the current object
        # input dim is the number of inpute features
        super(RewardModel, self).__init__() # calls the parent classe's constructor
        self.model = nn.Sequential( # sequential container for layers
            nn.Linear(input_dim, 32), # hidden layer 1, fully connected with 32 neurons
            nn.ReLU(), # applies the rectified linear unit activation function
            nn.Linear(32,16), # hidden layer 2 with 16 neurons
            nn.ReLU(), # activation for second layer
            nn.Linear(16,1))  # output layer with 1 neuron to predict reward score
    def forward(self, x): # forward defines how the input tensor moves through the model
        # x is the input feature vector
        return self.model(x) # feed x through the defined layers and return the result

In [8]:
# execute a notebook and return a specific variables output
def run_notebook(notebook_path, input_data, output_variable):
    with open(notebook_path) as f: # with is used to safelf open and close the file
        # open() opens the file at notebook path in read mode
        # as f assignes the file handle to variable f
        nb = nbformat.read(f, as_version=4) # read the notebook using version 4 format
    # inject the user input at the top of the notebook
    nb.cells.insert(0, nbformat.v4.new_code_cell(f"user_input = '''{input_data}'''"))
    # nb.cells.insert() inserts a new code cell at the beginning of the notebook
    # nbformat.v4.new_code_cell() creats a code cell with user input
    ep = ExecutePreprocessor(timeout=600) # create an execute preprocessor instance with a timeout of 600 seconds
    try:
        ep.preprocess(nb) # runs all the cells in the notebook
        # search for output variable in each cell
        for cell in nb.cells: # loop goes through each cell in the notebook
            if cell.cell_type == 'code' and 'outputs' in cell:
                # if the code cell contains code and output
                for output in cell.outputs:
                    # if the output contains text
                    if 'text' in outputs:
                        try:
                            result = json.loads(output.text) # parse output text as json
                            if output_variable in result:
                                # if the desired variable is in the result
                                return result[output_variable] # return that value
                        except:
                            continue # skip if json parsing fails
    except Exception as e:
        print(f"error in notebook execution: {e}") # print execution error
    return {} # return empty dictionary if no output found

In [9]:
# feature encoding: convert emotion, personality, and input to tensor vector
def encode_features(emotion, personality, user_input): # define possible emotions in fixed order for one hot encoding
    emotion_keys = ['happy', 'sad', 'angry', 'fear', 'neutral'] # defined emotion labels
    feature_vec.append = [] # list to store the encoded features
    # one hot encode the dominant emotion ([1,0,0,0,0] for happy)
    for emo in emotion_keys:
        feature_vec.append(1 if emo == emotion.get('dominant_emotion') else 0)
    # append personality trait scores in sorted order of trait names
    for trait in sorted(personality.keys()):
        feature_vec.append(personality[trait])
    # append the length of the user input, normalized by 100
    feature_vec.append(len(user_input)/100.0)
    return torch.tensor(feature_vec, dtype=torch.float32) # convert the list to a pytorch vector with type float32

In [10]:
# store user feedback data in a file for later model training
def store_feedback(user_input, response, emotion, personality, feedback_score):
    data = {  # create a dictionary with all relevant feedback data
        "user_input": user_input, # original input
        "response": response, # bots response
        "emotion_result": emotion, # emotion analysis result
        "personality_result": personality, # personality traits
        "user_feedback": feedback_score # users rating on the response
    }
    # append the data as json to a log file
    with open("feedback_logs.jsonl", "a") as f:
        # open the feedback file in append mode ("a")
        f.write(json.dumps(data) + "\n") # write data and add new line
        # json.dumps() convert the dictionary into a json string

In [11]:
# train the reward model using the stored feedback from the file
def train_reward_model(log_file="feedback_logs.jsonl"):
    features, labels = [], [] # lists to store input fetures and target labels
    # open the feedback file in read mode ("r")
    with open(log_file,"r") as f:
        for line in f: # read line by line
            item = json.loads(line) # parse each line as json
            x = enocde_features(item['emotion_result'], item['personality_result'], item['user_input']) # input features
            y = torch.tensor([item['user_feedback']], dtype=torch.float32) # target feedback score
            features.append(x)
            balels.append(y)
    # initialize the model with input size = feature length
    model = RewardModel(len(features[0]))
    criterion = nn.MSELoss() # loss function: mean squared error
    optimizer = optim.Adam(model.parameters(), lr=0.01) # optimizer: adam with learning rate 0.01
    # train for 10 epochs
    for epoch in range(10):
        total_loss = 0 # track total loss for each epoch
        for x,y in zip(features, labels): # loop through each training sample
            optimizer.zero_grad() # reset gradients before each update
            output = mode(x) # predict output using the model
            loss = criterion(output, y) # calculate the loss between predicted and actual feedback
            loss.backward() # backpropagate error
            optimizer.step() # update model parameters
            total_loss += loss.item() # accumulate loss
        print(f"epoch {epoch+1}, Loss: {total_loss:.4f}")
    return model

In [12]:
# generate a response based on detected emotion and personality
def generate_response(user_input, emotion, personality):
    mood = emotion.get("dominant_emotion", "neutral") # get dominant emotion from dictionary
    # coose tone based on agreeableness personality trait
    style = "friendly" if personality.get("agreeableness", 0.5) > 0.6 else "neutral"
    # returns a basic response string
    return f"i am sensing you are feeling {mood}. i am here with a {style} tone. tell me more?"

In [15]:
# main function to run the chatbot loop
def main():
    print("talk to me")
    while True: # infinite loop for conversation
        user_input = input("you: ")
        if user_input.lower() == "exit": # check for exit command
            break
        # run notebooks to get emotion and personality analysis
        emotion_result = run_notebook("EMTN_DTC.ipynb", user_input, "emotion_output")
        personality_result = run_notebook("PRSN_DTC.ipynb", user_input, "personality_traits")
        # generate a response using analysis results
        response = generate_response(user_input, emotion_result, personality_result)
        print("bot: ", response)
        try:
            # ask user for feedback and convert it to float
            feedback = float(input("rate response 1-5: "))
            # save feedback to file
            store_feedback(user_input, response, emotion_result, personality_result, feedback)
        except:
            print("invalid feedback") # handle non neumeric feed back

In [16]:
# entry point check to run main function
# __name__ is a built in variale that is __main__ when this file is run directly
if __name__ == "__main__":
    main() # start the chat bot

talk to me


you:  hello bot! hope you doing well today.


error in notebook execution: name 'outputs' is not defined
bot:  i am sensing you are feeling neutral. i am here with a neutral tone. tell me more?


rate response 1-5:  2
you:  exit
